In [1]:
#! pip install ipywidgets
#! jupyter nbextension enable --py widgetsnbextension --sys-prefix
from ipywidgets import interact
from ipywidgets.widgets import FloatText, BoundedIntText, FloatSlider, IntSlider, IntRangeSlider, fixed, Text

import numpy as np
import matplotlib.pyplot as plt

from typing import Tuple, Iterable

In [2]:
# Widgets for interactive functions / plots
style = {'description_width': 'initial'}

deposit_text = FloatText(value=1000, step=100, description='Deposit:')

contribution_text = FloatText(value=100, step=10, description='Yearly contribution:', style=style)

contribution_years_slider = IntSlider(value=0, step=1, min=0, max=100, description='Contribution years:', style=style)

interest_slider = FloatSlider(value=0.05, step=0.01, min=0, max=0.25, description='Interest:')
inflation_slider = FloatSlider(value=0.17, step=0.01, min=0, max=0.25, description='Inflation:')
interest_text = Text(value='0.10, 0.07', description="Interests")

tax_slider = FloatSlider(value=0.19, step=0.01, min=0, max=0.30, description='Tax:')

years_slider = IntSlider(value=1, step=1, min=1, max=100, description='Years:')
years_range_slider = IntRangeSlider(value=(0, 40), step=1, min=0, max=100, description='Years:')

frequency_slider = IntSlider(value=1, step=1, min=1, max=12, description='Frequency:')
frequencies_text = Text(value='2, 12', description="frequencies")

# Savings

 ### Time deposit
 Time deposits normally earn interest, which is normally fixed for the duration of the term and payable upon maturity, though some may be paid periodically during the term, especially with longer-term deposits. Generally, the longer the term and the larger the deposit amount the higher the interest rate that will be offered.

 source: https://en.wikipedia.org/wiki/Time_deposit

#### Certificate of deposit (lokata?)
CD has a specific, fixed term (often one, three, or six months, or one to five years) and usually, a fixed interest rate. The bank expects the CD to be held until maturity, at which time they can be withdrawn and interest paid. CDs are insured "money in the bank" (in the EU up to 100 000 euros) and thus, up to the local insured deposit limit, virtually risk free.
https://en.wikipedia.org/wiki/Certificate_of_deposit

In real world scenarios we need to add in taxes, which are took from your earned interest. Given initial deposit $D_0$ after $n$ periods with $p\%$ periodic interest rate, $s\%$ tax fee and $m=1$ frequency of compounding, i.e. $D_n$ is equal to
$D_n = D_{n-1} + D_{n-1} p\% (1 - s\%) = D_{n-1} \left( 1 + p\% (1 - s\%) \right) $

Similarly, when we take any $m$:
$D_n = D_{n-1} \left( 1 + \frac{p\%}{m} (1 - s\%) \right)^m $

We can see that adding tax is actually just a change of percentage, so we can denote $r = p\% (1 - s\%)$ to simplify. With that we see that tax is not changing the theorem proved before, so:
$D_n = D_0 \left( 1 + \frac{r}{m} \right)^{mn}$

In [3]:
def deposit_years(initial_deposit: float, years: int, frequency: int, interest: float, tax: float = 0.19):
    """Compute deposit amount after number of years."""
    return initial_deposit * (1 + interest * (1 - tax) / frequency)**(years * frequency)

def deposit(initial_deposit: float, n: int, frequency: int, interest: float, tax: float = 0.19):
    """Compute deposit amount after n compound periods."""
    return initial_deposit * (1 + interest * (1 - tax) / frequency)**n

### Deposit calculators

In [4]:
def pretty_print_deposits(initial_deposit, years, frequency, interest, tax):
    return f'Deposit after {years} years, with frequency {frequency}, {interest} interest rate and tax {tax}: {deposit_years(initial_deposit, years, frequency, interest, tax)}'

interact(pretty_print_deposits, initial_deposit=deposit_text, years=years_slider, frequency=frequency_slider, interest=interest_slider, tax=tax_slider);

interactive(children=(FloatText(value=1000.0, description='Deposit:', step=100.0), IntSlider(value=1, descript…

In [5]:
def plot_deposits(initial_deposit: float, years: Tuple[int, int], frequencies: str, interests: str, tax: float = 0.19):
    """Plot deposit amount as a function of periods for different compounding frequencies."""
    years = np.arange(*years)
    frequencies = list(map(int, frequencies.split(',')))
    interests = list(map(float, interests.split(',')))
    for m, p in zip(frequencies, interests):
        plt.plot(years, deposit_years(initial_deposit, years, m, p, tax))
    plt.xlabel('years')
    plt.ylabel('amount')
    plt.title(f'Initial deposit: {initial_deposit}')
    plt.legend(labels=list(zip(frequencies, interests)), title='Frequency and interest')
    plt.show()

interact(plot_deposits, initial_deposit=deposit_text, years=years_range_slider, frequencies=frequencies_text, interests=interest_text, tax=tax_slider);

interactive(children=(FloatText(value=1000.0, description='Deposit:', step=100.0), IntRangeSlider(value=(0, 40…

### Savings account
Savings accounts are more flexible CDs usually pay interest monthly, but have limited number of withdrawals.

https://en.wikipedia.org/wiki/Savings_account

### Time deposits with contributions
In real world scenario we will contribute to the deposit with each compounding period. Given initial deposit $D_0$ and the same amount of contribution for each compounding, after $n$ periods with $p\%$ periodic interest rate, $s\%$ tax fee and $m=1$ frequency of compounding, i.e. $D_n$ is equal to

\begin{align*}
    r &= p\% (1-s\%) \\
    q &= 1 + r \\
    D_n &= D_{n-1} (1 + r) + D_0 = D_{n-1}q + D_0
\end{align*}

Let's look at few first terms:
\begin{align*}
    D_0 &= D_0 \\
    D_1 &= D_0 q + D_0 \\
    D_2 &= D_1 q + D_0  = D_0 q^2 + D_0 q + D_0 \\
    D_3 &= D_2 q + D_0  = D_0 q^3 + D_0 q^2 + D_0 q + D_0 \\
    \vdots
\end{align*}
so our conjecture is that $D_n = D_0 \sum_{i=0}^{n} q^i$.

##### Theorem
$D_n = D_0 \sum_{i=0}^{n} q^i$

##### Proof by induction
For $n=1$ we have $D_1 = D_0 q + D_0$ by definition.

IH: $D_n = D_0 \sum_{i=0}^{n} q^i$

IS: $D_{n+1} = D_n q + D_0 = \left( D_0 \sum_{i=0}^{n} q^i \right)q + D_0 = D_0 \left[ \sum_{i=1}^{n+1} q^i + 1 \right] = D_0 \left[ \sum_{i=0}^{n+1} q^i \right]$ and that finishes the proof.

##### Corollary
$D_n = D_0 \cdot \frac{1 - (1 + r)^{n+1}}{-r}.$

##### Proof
We know the formula for geometric series: $\sum_{i=0}^{n} q^i = \frac{1-q^{n+1}}{1-q} = \frac{1 - (1 + r)^{n+1}}{-r}.$

For any compounding frequency $m$ we need to add more equations:

\begin{align*}
    D_{n-1, m} = D_{n,0} &= D_n \\
    D_{n, 1} &= D_{n,0}q + D_0 \\
    D_{n, 2} &= D_{n,1}q + D_0 = D_n q^2 + D_0 q + D_0  \\
    D_{n, 3} &= D_{n,2}q + D_0 = D_n q^3+  D_0 q^2 + D_0 q + D_0  \\
    \vdots \\
    D_{n, m-1} &= D_{n,m-2}q + D_0  = D_n q^{m-1} + D_0 \sum_{i=0}^{m-2} q^i \\
    D_{n+1} = D_{n,m} &= D_{n,m-1}q + D_0 = D_n q^{m} + D_0 \sum_{i=0}^{m-1} q^i
\end{align*}

Let's try substituting some first few equations:
\begin{align*}
    D_0 &= D_0 \\
    D_1 &= D_0 q^m + D_0 \sum_{i=0}^{m-1} q^i = D_0 \sum_{i=0}^{m} q^i\\
    D_2 &= D_1 q^m + D_0 \sum_{i=0}^{m-1} q^i = D_0 \sum_{i=m}^{2m} q^i + D_0 \sum_{i=0}^{m-1} q^i = D_0 \sum_{i=0}^{2m} q^i  \\
    D_3 &= D_2 q^m + D_0 \sum_{i=0}^{m-1} q^i = D_0 \sum_{i=m}^{3m} q^i + D_0 \sum_{i=0}^{m-1} q^i = D_0 \sum_{i=0}^{3m} q^i  \\
    \vdots
\end{align*}
so our conjecture is that $D_n = D_0 \sum_{i=0}^{nm} q^i$

##### Theorem
$D_n = D_0 \sum_{i=0}^{nm} q^i$

##### Proof by induction
For $n=1$ we have $D_1 = D_0 q^m + D_0 \sum_{i=0}^{m-1} = D_0 \sum_{i=0}^{m}$ by definition.

IH: $D_n = D_0 \sum_{i=0}^{nm} q^i$

IS: $D_{n+1} = D_n q^m + D_0 \sum_{i=0}^{m-1} = \left( D_0 \sum_{i=0}^{nm} q^i \right) q^m + D_0 \sum_{i=0}^{m-1} q^i = D_0 \sum_{i=m}^{(n+1)m} q^i + D_0 \sum_{i=0}^{m-1} q^i = D_0 \sum_{i=0}^{(n+1)m} q^i$ and that finishes the proof.

##### Corollary
$D_n = D_0 \cdot \frac{1 - (1 + r)^{nm+1}}{-r}.$

##### Proof
We know the formula for geometric series: $\sum_{i=0}^{nm} q^i = \frac{1-q^{nm+1}}{1-q} = \frac{1 - (1 + r)^{nm+1}}{-r}.$

It is easy to see and prove by induction, that if we have initial deposit $D_0$ and different contribution per compound $c$ then the general formula changes to:
$D_n = D_0 q^{nm} + c \sum_{i=0}^{nm-1} q^i$ and $D_n = D_0 q^{nm} + c \cdot \frac{1 - q^{nm}}{1-q}$

In [6]:
def contribute_deposit_years(initial_deposit: float, years: int, frequency: int,
                  interest: float, tax: float = 0.19, contribution: float = 0, contrib_years: int = 0):
    """Calculate deposit amount after number of years."""
    if contribution == 0: # not contributing, then exponential function
        D_n = initial_deposit * (1 + interest * (1 - tax) / frequency)**(years * frequency)
        return D_n
    else: # contributing then geometric series
        contribution = contribution / frequency
        r = interest / frequency * (1-tax)
        c_y = contrib_years # years with contributing
        n_y = years - contrib_years # years without contributing
        D_c_y = (initial_deposit
               * (1 + r)**(c_y * frequency)
               + contribution * (1 - (1+r)**(c_y*frequency)) / (-r) ) # deposit after years of contribution
        D_n = deposit_years(D_c_y, years=n_y, frequency=frequency, interest=interest, tax=tax) # deposit after years without contributing
        return D_n

def contribute_deposit(initial_deposit: float, n: int, frequency: int, interest: float,
            tax: float = 0.19, contribution: float = 0, contrib_periods: int = 0):
    """Calculate deposit amount after n compound periods."""
    if contribution == 0: # not contributing, then exponential function
        D_n = initial_deposit * (1 + interest * (1 - tax) / frequency)**n
        return D_n
    else: # contributing then geometric series
        r = interest / frequency * (1-tax)
        contribution /= frequency
        func = lambda x: contrib_periods if x >= contrib_periods else x
        c_n =  np.array(list(map(func, n))) # periods with contributing
        n_n = n - contrib_periods # periods without contributing
        D_c_y = (initial_deposit
                 * (1 + r)**c_n
                 + contribution * (1 - (1+r)**c_n) / (-r) ) # deposit after periods of contribution
        D_n = deposit(D_c_y, n=n_n, frequency=frequency, interest=interest, tax=tax) # deposit after periods without contributing
        return D_n

d1 = contribute_deposit_years(initial_deposit=2000, years=65-22, frequency=1, interest=0.06, tax=0.19, contrib_years=39-22, contribution=2000)
d2 = contribute_deposit_years(initial_deposit=2000, years=66-31, frequency=1, interest=0.06, tax=0.19, contrib_years=66-31, contribution=2000)

In [7]:
def deposit_test(initial_deposit: float, n: int, frequency: int, interest: float,
                 tax: float = 0.19, contribution: float = 0, contrib_periods: int = 0):
    """Compute deposit amount after n compound periods."""
    d_n = initial_deposit
    r = interest / frequency * (1-tax)
    for i in range(contrib_periods):
        d_n = d_n * (1+r) + contribution
        print(f'period={i+1} with contribution, deposit={d_n:.2f}')
    for i in range(n - contrib_periods):
        d_n = d_n * (1+r)
        print(f'period={i+1} without contribution, deposit={d_n:.2f}')
    return d_n

#deposit_test(initial_deposit=2000, n=65-22, frequency=1, interest=0.06, tax=0, contrib_periods=39-22, contribution=2000)
#deposit_test(initial_deposit=2000, n=66-31, frequency=1, interest=0.06, tax=0, contrib_periods=66-31, contribution=2000)
#deposit_test(initial_deposit=1000, n=20*12, frequency=12, interest=0.05, tax=0.19, contrib_periods=10*12, contribution=1000/12)
#deposit_years(initial_deposit=1000, years=20, frequency=12, interest=0.05, tax=0.19, contrib_years=10, contribution=1000)

### Calculators and plots

In [8]:
def plot_deposit(initial_deposit: float, years: Tuple[int, int], frequencies: str, interests: str,
                 tax: float = 0.19, contribution: float = 0, contrib_years: int = 0):
    """Plot deposit amount as a function of periods for different compounding frequencies."""
    years_range = np.arange(1, years[1]+1)
    frequencies = list(map(int, frequencies.split(',')))
    interests = list(map(float, interests.split(',')))
    for m, p in zip(frequencies, interests):
        d_n = deposit_years(initial_deposit, years_range, m, p, tax, contribution, contrib_years)
        plt.plot(years_range, d_n)
    plt.xlabel('years')
    plt.ylabel('amount')
    plt.xlim(*years)
    plt.title(f'Investment: {initial_deposit+contribution*contrib_years:.2f}')
    plt.legend(labels=list(zip(frequencies, interests)), title='Frequency and interest')
    plt.show()

interact(plot_deposit, initial_deposit=deposit_text, years=years_range_slider, frequencies=frequencies_text,
         interests=interest_text, tax=tax_slider, contribution=contribution_text, contrib_years=contribution_years_slider);

interactive(children=(FloatText(value=1000.0, description='Deposit:', step=100.0), IntRangeSlider(value=(0, 40…

## Inflation
We know that prices are going up with each year and we call this factor an inflation (let's denote it as $f$). If the prices are twice as much as last year, then $f=100\%$. It affects your deposit, because you can buy twice less. What is the relation between your deposit and future worth of it? Let's say that we have $D_n$ deposit at $n$-th period, we can buy $m$ goods for a price of $c_n$ and the rate of inflation at this period is $f$, then:

\begin{cases}
    D_n &= m \cdot c_n, \\
    c_{n+1} &= (1+f) c_n.
\end{cases}

If we want to uphold this equality in the next period we need to increase our deposit:
\begin{equation}
    D_n &= m \cdot \frac{c_{n+1}}{1+f}, \\
    (1+f) D_n &= m \cdot c_{n+1}, \\
    D_{n+1} &= m \cdot c_{n+1}.
\end{equation}

So in the next period we need to have $D_{n+1} = (1+f) D_n$. It means, that we need more money, to buy the same amount of things.

##### Example
Given 100 zł, with yearly inflation rate of 10\% , next year you will need 110 zł to buy the same amount of things.

You have 100 zł, what will the worth of it next year with the same parameters as before? We have to reverse equation, as we are now asking about future $D_{n+1} = 100$ and want to know the value in present so $D_n$:
\begin{equation}
    D_{n} = \frac{D_{n+1}}{1+f} = \frac{100}{1.1} = 90,91.
\end{equation}

It is easy to see that with the formula is:
\begin{equation}
    D_{n-1} = D_n\left( 1+f \right)^{-1}.
\end{equation}

### Compound frequency
 Let $D_N$ be the last deposit, $f\%$ periodic inflation rate, $m$ frequency of compounding and denote $D_{n,k}, \; k=0,\ldots, m$ as deposit amount in $n$-th period and $k$-th compounding. Also for convenience denote $D_n = D_{n,0} = D_{n-1, m}, \; D_{N,0} = D_N$ as deposit after $n$ periods.

\begin{cases}
    D_{n,m} & = D_{n+1} \\
    D_{n,m-1} & = D_{n,m} \left( 1 + \frac{f}{m}\% \right)^{-1} = D_{n+1} \left( 1 + \frac{f}{m}\% \right)^{-1} \\
    D_{n,m-2} & = D_{n,m-1} \left( 1 + \frac{f}{m}\% \right)^{-1} = D_{n+1} \left( 1 + \frac{f}{m}\% \right)^{-2} \\
    \vdots \\
    D_{n,1} & = D_{n, 2} \left( 1 + \frac{f}{m}\% \right)^{-1} = D_{n+1} \left( 1 + \frac{f}{m}\% \right)^{-(m-1)} \\
    D_{n,0} & = D_{n, 1} \left( 1 + \frac{f}{m}\% \right)^{-1} = D_{n+1} \left( 1 + \frac{f}{m}\% \right)^{-(m-0)}
\end{cases}

We can see that $D_{n,k} = D_{n+1} \left( 1 + \frac{f}{m}\% \right)^{-(m-k)}$. Substituting equations from top to the bottom we will get that $D_{n} = D_{n+1} \left( 1 + \frac{f}{m}\% \right)^{-m}$

##### Theorem
The deposit amount in $n$-th period ($n \in \left[0, N \right]$), given last deposit amount $D_N$ (after $N$ periods) with $f\%$ with inflation rate and $m$ frequency of compounding, i.e. $D_n$ is equal to $D_N \left( 1 + \frac{f}{m}\% \right)^{-m(N-n)}$.

##### Proof by induction
For $n=N$ we have: $D_N = D_N \left( 1 + \frac{f}{m}\% \right)^{-m \cdot 0}$ by definition.

IH: $D_n = D_N \left( 1 + \frac{f}{m}\% \right)^{-m(N-n)}$

IS: $D_{n-1} = D_n \left( 1 + \frac{f}{m}\% \right)^{-m} = D_N \left( 1 + \frac{f}{m}\% \right)^{-m(N-n)} \left( 1 + \frac{f}{m}\% \right)^{-m} = D_N \left( 1 + \frac{f}{m}\% \right)^{-m(N-(n-1))}$ and that finishes the proof.


### Continuous inflation
It seems that inflation is continuous process? Is it because, there are no discrete jumps between price changes?
$D_t = \lim_{m \rightarrow \infty} D_N \left( 1 + \frac{f}{m}\% \right)^{-m(N-t)} = D_N \left\{  \lim_{m \rightarrow \infty} \left[ \left( 1 + \frac{1}{\frac{m}{f\%}} \right)^{\frac{m}{f\%}} \right]^{f\%} \right\}^{-(N-t)} = D_N e^{-f\%(N-t)}$

## Real certificate of deposit
In real world scenario inflation always exists, so the money is always depreciating and in usually we lose money instead of earning it in CDs. We can calculate our gains/losses by first applying deposits formula so we know how much we earned and then use inflation formula to get the actual worth of future money in present money terms.

Given initial deposit $D_0$ with $p\%$ periodic interest rate, $s\%$ tax fee and $m$ frequency of compounding, we calculate amount of deposit after $N$ periods.
\begin{equation}
    D_N = D_0 \left( 1 + \frac{p\%}{m} (1 - s\%) \right)^{mN} =  D_0 \left( 1 + \frac{r}{m} \right)^{mN}.
\end{equation}

Now that we have future value, we can apply inflation formula to get present value $D_0'$ in present money value terms (inflation is a continuous process, but we can simplify it by using discrete approximation, which is really close):
\begin{equation}
    D_0' = D_N \left( 1 + \frac{f}{m} \right)^{-mN} = D_0 \left( 1 + \frac{r}{m} \right)^{mN} \left( 1 + \frac{f}{m} \right)^{-mN} = D_0 \left( \frac{1 + \frac{r}{m}}{1 + \frac{f}{m}} \right)^{mN} = D_0 \left( \frac{m+r}{m+f} \right)^{mN} = D_0 \left( 1 + \frac{r-f}{m+f} \right)^{mN}.
\end{equation}

It is easy to see that if $f=r$ our initial deposit stays the same.

In [9]:
def real_deposit(initial_deposit: float, n: int, frequency: int, interest: float, inflation: float, tax: float = 0.19):
    return initial_deposit * ( (frequency + interest*(1-tax))/(frequency + inflation) )**n

real_deposit(initial_deposit=50_000, n=1, frequency=2, interest=.1, inflation=0.17) - 50_000

-2050.691244239635

### Real CD with contribution
We just need to apply different formula for deposit with contributions from before:
\begin{equation}
    D_N = D_0 (1+r)^{mN} + c \cdot \frac{1 - (1+r)^{mN}}{-r},
\end{equation}

and apply inflation formula:
\begin{equation}
    D_0' = D_N \left( 1 + \frac{f}{m} \right)^{-mN} = \left[ D_0 (1+r)^{mN} + c \cdot \frac{1 - (1+r)^{mN}}{-r} \right] \left( 1 + \frac{f}{m} \right)^{-mN}.
\end{equation}


In [10]:
def inflate(last_deposit: float, N: int, frequency: int, inflation: float):
    return last_deposit / (1 + inflation / frequency)**N

def real_deposit(initial_deposit: float, n: int, frequency: int, interest: float, inflation: float,
                 tax: float = 0.19, contribution: float = 0, contrib_periods: int = 0):
    """Calculate deposit amount after n compound periods."""
    d_n =  contribute_deposit(initial_deposit=initial_deposit, n=n, frequency=frequency, interest=interest, tax=tax, contribution=contribution, contrib_periods=contrib_periods)
    return inflate(last_deposit=d_n, N=n, frequency=frequency, inflation=inflation)

real_deposit(initial_deposit=50_000, n=1, frequency=2, interest=.1, inflation=0.17) - 50_000

-2050.691244239628

In [13]:
def plot_real_deposit(initial_deposit: float, years: Tuple[int, int], frequencies: str, interests: str, inflation: float,
                      tax: float = 0.19, contribution: float = 0, contrib_years: int = 0):
    """Plot deposit amount as a function of periods for different compounding frequencies."""
    years_range = np.arange(0, years[1] + 1)
    frequencies = list(map(int, frequencies.split(',')))
    interests = list(map(float, interests.split(',')))
    for m, p in zip(frequencies, interests):
        d_n = real_deposit(initial_deposit=initial_deposit, n=m*years_range, frequency=m, interest=p,
                           inflation=inflation, tax=tax, contribution=contribution, contrib_periods=m*contrib_years)
        print(d_n)
        plt.plot(years_range, d_n)
    plt.xlabel('years')
    plt.ylabel('amount')
    plt.xlim(*years)
    plt.title(f'Investment: {initial_deposit+contribution*contrib_years:.2f}')
    plt.legend(labels=list(zip(frequencies, interests)), title='Frequency and interest')
    plt.show()

interact(plot_real_deposit, initial_deposit=deposit_text, years=years_range_slider, frequencies=frequencies_text, inflation=inflation_slider,
         interests=interest_text, tax=tax_slider, contribution=contribution_text, contrib_years=contribution_years_slider);

interactive(children=(FloatText(value=1000.0, description='Deposit:', step=100.0), IntRangeSlider(value=(0, 14…